# Modeling tuning
using random search several times to find best results with different parameters. 



In [ ]:
import pandas as pd
import numpy as np
#import seaborn as sns
import random
import pickle

In [ ]:

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:

import matplotlib.pyplot as plt
from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA

In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [6]:
# from google.colab import drive
# drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# from google.colab import files

# #define function to save models locally after they have run
# def save_model(model, string):
#   filename = string + '.sav'
#   pickle.dump(model, open(filename, 'wb'))
#   files.download(filename)
#print(filename, ' has been saved to drive!')

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Preprocessing 

In [ ]:
#import file
data= pd.read_csv('gdrive/My Drive/Colab Notebooks/county_df.csv')
data.drop(columns = "smallbrew_per_capita", inplace = True)

In [ ]:
data.set_index(['Area','County','Year'], inplace = True)
#convert all to numeric
data = data.apply(pd.to_numeric, errors = 'coerce')
#save dataframe for later just i
save = data


In [ ]:
data.reset_index(inplace = True)

In [ ]:

from sklearn.preprocessing import QuantileTransformer, OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
#from sklearn.imputer import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, mean_absolute_error
def adjusted_r2(train, r2):
  adj_r2 = 1 - (1 - r2) * ((train.shape[0] - 1) / (train.shape[0] - train.shape[1] - 1))
  return adj_r2



### Random Forests

In [ ]:

#first try
data_y = data.Breweries_Small
data_X = data.drop(['Breweries_Small', 'Area'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=42)

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)
y_pred = forest_reg.predict(X_test)

In [ ]:
#calculate metrics 

r2 = r2_score(y_test, y_pred)
adj_r2 = adjusted_r2(X_train, r2)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

#results.append(['forest','none', adj_r2, rmse, mae])

print('adjusted r2: %.2f' % adj_r2)
print('RMSE: %.2f' % rmse)
print('MAE: %.2f' % mae)



adjusted r2: 0.81
RMSE: 1.24
MAE: 0.60


#### random search 1 - 4

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, 
                                                    random_state=42)

forest_reg = RandomForestRegressor()

In [ ]:


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 15)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_search = RandomizedSearchCV(forest_reg, random_grid,n_iter = 50, cv = 3, verbose=5, random_state=42, n_jobs = -1)

In [ ]:
random_search.fit(X_train, y_train)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 12.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=50, n_jobs=-1,
          param_distributions={'n_estimators': [20, 32, 45, 58, 71, 84, 97, 110, 122, 135, 148, 161, 174, 187, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [2, 4, 6], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=5)

In [ ]:
estimator1 = random_search.best_estimator_

In [ ]:
def evaluate_model(model, X_test, y_test):
  y_pred = model.predict(X_test)
  r2 = r2_score(y_test, y_pred)
  adj_r2 = adjusted_r2(X_train, r2)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(y_test, y_pred)
   
  
  print('Model Performance')

  print('adjusted r2: %.2f' % adj_r2)
  print('RMSE: %.2f' % rmse)
  print('MAE: %.2f' % mae)

In [ ]:
evaluate_model(estimator1, X_test, y_test)

Model Performance
adjusted r2: 0.80
RMSE: 1.27
MAE: 0.58


In [ ]:
random_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=80,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=97, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
## random search 2 ##

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 150, num = 10)]
#number of components for pca
#n_components = [5, 10, 15, 20, 30, 40, 90]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_search2 = RandomizedSearchCV(forest_reg, random_grid,n_iter = 30, 
                                    cv = 3, verbose=3, random_state=42, n_jobs = -1)

In [ ]:
random_search2.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   58.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  5.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'n_estimators': [10, 25, 41, 56, 72, 87, 103, 118, 134, 150], 'max_features': ['auto', 'sqrt'], 'max_depth': [50, 65, 80, 95, 110, None], 'min_samples_split': [2, 3, 5, 10], 'min_samples_leaf': [2, 4, 6], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=3)

In [ ]:
estimator2 = random_search2.best_estimator_

In [ ]:
evaluate_model(estimator2, X_test, y_test)

Model Performance
adjusted r2: 0.81
RMSE: 1.23
MAE: 0.57


In [ ]:
estimator2

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=72, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
## random search 3 ##

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 100, num = 5)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(30, 80, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3, 5, 7,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_search3 = RandomizedSearchCV(forest_reg, random_grid,n_iter = 30, 
                                    cv = 3, verbose=3, random_state=42, n_jobs = -1)

In [ ]:
random_search3.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'n_estimators': [20, 40, 60, 80, 100], 'max_features': ['sqrt'], 'max_depth': [30, 42, 55, 67, 80, None], 'min_samples_split': [2, 3, 5, 7, 10], 'min_samples_leaf': [2, 4, 6], 'bootstrap': [False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=3)

In [ ]:
estimator3 = random_search3.best_estimator_
evaluate_model(estimator3, X_test, y_test)

Model Performance
adjusted r2: 0.80
RMSE: 1.25
MAE: 0.57


In [ ]:
estimator3

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=42,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
## random search 4 ##

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 90, num = 5)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(30, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3, 5, 7,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_search4 = RandomizedSearchCV(forest_reg, random_grid,n_iter = 30, 
                                    cv = 3, verbose=3, random_state=42, n_jobs = -1)

In [ ]:
random_search4.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'n_estimators': [30, 45, 60, 75, 90], 'max_features': ['sqrt'], 'max_depth': [30, 37, 45, 53, 61, 68, 76, 84, 92, 100, None], 'min_samples_split': [2, 3, 5, 7, 10], 'min_samples_leaf': [2, 4, 6], 'bootstrap': [False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=3)

In [ ]:
estimator4 = random_search4.best_estimator_
evaluate_model(estimator4, X_test, y_test)

Model Performance
adjusted r2: 0.80
RMSE: 1.26
MAE: 0.58


### Using Onehotencoder
try one hot encoder to make states into a categorical feature and see if there are better results

In [ ]:
data_y = data.Breweries_Small
data_X = data.drop('Breweries_Small', axis = 1)

states = data_X.Area.unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
le.fit(states)
data_X.Area = le.transform(data_X.Area)


pca = PCA(n_components = 20)


In [ ]:
categorical_features = ['Area']
categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

ct = ColumnTransformer(
    transformers=[
       
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
data_X = ct.fit_transform(data_X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, 
                                                    random_state=42)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 150, num = 10)]
#number of components for pca
#n_components = [5, 10, 15, 20, 30, 40, 90]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_search5 = RandomizedSearchCV(forest_reg, random_grid,n_iter = 30, 
                                    cv = 3, verbose=3, random_state=42, n_jobs = -1)

In [ ]:
random_search5.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   18.6s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'n_estimators': [10, 25, 41, 56, 72, 87, 103, 118, 134, 150], 'max_features': ['auto', 'sqrt'], 'max_depth': [50, 65, 80, 95, 110, None], 'min_samples_split': [2, 3, 5, 10], 'min_samples_leaf': [2, 4, 6], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=3)

In [ ]:
estimator5 = random_search5.best_estimator_
evaluate_model(estimator5, X_test, y_test)

Model Performance
adjusted r2: 0.09
RMSE: 2.70
MAE: 1.15


Results are much worse...

### Using  PCA
try PCA to see if it produces a better result

In [ ]:
data_y = data.Breweries_Small
data_X = data.drop(['Breweries_Small' ,'Area'], axis = 1)




pca = PCA(n_components = 20)
data_X = pca.fit_transform(data_X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, 
                                                    random_state=42)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 150, num = 10)]
#number of components for pca
#n_components = [5, 10, 15, 20, 30, 40, 90]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_search6 = RandomizedSearchCV(forest_reg, random_grid,n_iter = 30, 
                                    cv = 3, verbose=3, random_state=42, n_jobs = -1)

In [ ]:
random_search6.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   51.9s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  4.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'n_estimators': [10, 25, 41, 56, 72, 87, 103, 118, 134, 150], 'max_features': ['auto', 'sqrt'], 'max_depth': [50, 65, 80, 95, 110, None], 'min_samples_split': [2, 3, 5, 10], 'min_samples_leaf': [2, 4, 6], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=3)

In [ ]:
estimator6 = random_search6.best_estimator_
evaluate_model(estimator6, X_test, y_test)

Model Performance
adjusted r2: 0.73
RMSE: 1.46
MAE: 0.65


Slightly worse than with out... won't use. 

#### Random Search 6 - 9
without onehot or PCA

In [ ]:
estimator6

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=80,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=134, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:

data_y = data.Breweries_Small
data_X = data.drop(['Breweries_Small' ,'Area'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, 
                                                    random_state=42)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 120 , num = 5)]
#number of components for pca
#n_components = [5, 10, 15, 20, 30, 40, 
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_search9 = RandomizedSearchCV(forest_reg, random_grid,n_iter = 100, 
                                    cv = 5, verbose=3, random_state=42, n_jobs = -1)

In [36]:
random_search9.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  7.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [30, 52, 75, 97, 120], 'max_features': ['sqrt'], 'max_depth': [50, 65, 80, 95, 110, None], 'min_samples_split': [2, 3, 5, 10, 15], 'min_samples_leaf': [2, 4, 6, 8], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=3)

In [39]:
estimator9= random_search9.best_estimator_
print(estimator7)
print(estimator8)
print(estimator9)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=80,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=80,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=50,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_

In [37]:
evaluate_model(estimator8, X_test, y_test)

Model Performance
adjusted r2: 0.81
RMSE: 1.23
MAE: 0.57


In [40]:
evaluate_model(estimator9, X_test, y_test)

Model Performance
adjusted r2: 0.80
RMSE: 1.25
MAE: 0.57


Estimator 8 is the best model. 